In [1]:
import torch

In [2]:
#chargeaons nos données
from dataloader import image_loader

#extrait data 1 à 1
train_dataset = image_loader(annotations_file='/mounts/Datasets4/GeoLifeCLEF2022/observations/observations_fr_train.csv', img_dir='patches-fr', transform=None, target_transform=None, tipe='train')

valid_dataset = image_loader(annotations_file='/mounts/Datasets4/GeoLifeCLEF2022/observations/observations_fr_train.csv', img_dir='patches-fr', transform=None, target_transform=None, tipe='val')

In [3]:
from torch.utils.data import DataLoader

#on crée le dataloader

batch_size = 64
shuffle = 'false'

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=shuffle)


In [4]:
from model import SimpleCNN

# Créez une instance du modèle
model = SimpleCNN()

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Définissez votre fonction de perte (loss function) et l'optimiseur Adam.
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Vous pouvez ajuster le taux d'apprentissage (lr) selon vos besoins.

# Définissez le nombre d'époques d'entraînement souhaité.
num_epochs = 10

for epoch in range(num_epochs):
    # Mode d'entraînement
    model.train()
    for batch in train_loader:
        images, labels = batch

        # Fusionner les 4 images pour chaque échantillon dans le batch
        images = images.view(-1, 12, 256, 256)  # Fusion des canaux des 4 images

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Mettez à jour la barre de chargement
        train_loader.set_postfix(loss=loss.item())  # Affichez la perte actuelle dans la barre de chargement

    # Mode d'évaluation (validation)
    model.eval()
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for batch in valid_loader:
            images, labels = batch  # Obtenez les images et les étiquettes du lot

            # Passez les données à travers le modèle.
            outputs = model(images)

            # Obtenez les 30 classes les plus probables pour chaque échantillon.
            _, top30_preds = torch.topk(outputs, 30, dim=1)

            # Vérifiez si la vraie étiquette se trouve dans les 30 prédictions les plus probables.
            correct_batch = [(true_label in top30_pred) for true_label, top30_pred in zip(labels, top30_preds)]

            # Calculez le nombre de prédictions correctes dans ce lot.
            total_correct += sum(correct_batch)
            total_samples += len(correct_batch)

    # Calculez la précision de validation.
    accuracy = total_correct / total_samples

    # Affichez les résultats de l'époque.
    print(f'Époque [{epoch + 1}/{num_epochs}], Perte d\'entraînement: {loss.item():.4f}, Précision de validation: {accuracy:.2%}')


IndexError: Target 674 is out of bounds.